In [ ]:
!pip install torch torchvision
!pip install numpy
!pip install open3d==0.16

In [ ]:
!pip show torch torchvision numpy open3d

Name: torch
Version: 2.0.0+cu118
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.9/dist-packages
Requires: filelock, jinja2, networkx, sympy, triton, typing-extensions
Required-by: fastai, MinkowskiEngine, torchaudio, torchdata, torchtext, torchvision, triton
---
Name: torchvision
Version: 0.15.1+cu118
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.9/dist-packages
Requires: numpy, pillow, requests, torch
Required-by: fastai
---
Name: numpy
Version: 1.22.4
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /usr/local/lib/

In [ ]:
!git clone https://github.com/guatavita/FCGF.git

Cloning into 'FCGF'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 289 (delta 95), reused 86 (delta 75), pack-reused 158
Receiving objects: 100% (289/289), 3.43 MiB | 10.20 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
%cd FCGF

/content/FCGF


In [ ]:
# !sudo apt install libopenblas-dev

In [ ]:
!pip install git+https://github.com/NVIDIA/MinkowskiEngine.git --install-option="--blas=openblas" -v

Using pip 22.0.4 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/MinkowskiEngine.git to /tmp/pip-req-build-q89l2_u6
  Running command git version
  git version 2.25.1
  Running command git clone --filter=blob:none https://github.com/NVIDIA/MinkowskiEngine.git /tmp/pip-req-build-q89l2_u6
  Cloning into '/tmp/pip-req-build-q89l2_u6'...
  Updating files:   0% (2/244)
  Updating files:   1% (3/244)
  Updating files:   2% (5/244)
  Updating files:   3% (8/244)
  Updating files:   4% (10/244)
  Updating files:   5% (13/244)
  Updating files:   6% (15/244)
  Updating files:   7% (18/244)
  Updating files:   8% (20/244)
  Updating files:   9% (22/244)
  Updating files:  10% (25/244)
  Updating files:  11% (27/244)
  Updating files:  12% (30/244)
  Updating files:  13% (32/244)
  Updating files:  14% (35/244)
  Updating files:  15% (37/244)
  Updati

In [ ]:
import torch
torch.__version__ # should be '1.12.1+cu113'
import MinkowskiEngine as ME
ME.__version__ # should be '0.5.4'

'0.5.4'

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pwd
%cd /content/FCGF

/content
/content/FCGF


In [ ]:
import open3d as o3d
print(o3d.__version__)

0.17.0


In [ ]:
import sys
sys.path.append('/content/FCGF/util/visualization.py')
sys.path.append('/content/FCGF/util/misc.py')
sys.path.append('/content/FCGF/model/resunet.py')

In [ ]:
import open3d as o3d
import numpy as np
import torch
from typing import Union
import plotly.graph_objs as go

In [ ]:
!python demo.py

python3: can't open file '/content/demo.py': [Errno 2] No such file or directory


In [ ]:
pcd = o3d.io.read_point_cloud('/content/FCGF/redkitchen-20.ply')
o3d.visualization.draw_plotly([pcd], o3d.visualization.draw_plotly([pcd], point_sample_factor=0.1))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Load point cloud from file
pcd = o3d.io.read_point_cloud('/content/FCGF/processed_cloud.ply')

# Down-sample point cloud
pcd_down = pcd.voxel_down_sample(voxel_size=0.01)

# Visualize down-sampled point cloud
o3d.visualization.draw_plotly([pcd_down], point_sample_factor=0.1)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# the function takes in two inputs, points and transformation, and applies the transformation matrix to the points.
def apply_transformation(
        points: Union[np.ndarray, torch.Tensor],
        transformation: Union[np.ndarray, torch.Tensor],
) -> Union[np.ndarray, torch.Tensor]:
    """
    :param points: tensor of shape (N, 3) representing floating point coordinates
    :param transformation: (4, 4) tensor of a transformation matrix
    :return: transformed points
    """
    if all(isinstance(i, np.ndarray) for i in [points, transformation]):
        transformed_points = np.matmul(
            transformation,
            np.concatenate(
                [points[:, 0:3], np.ones(shape=(points.shape[0], 1))], axis=-1
            ).T,
        ).T
    elif all(isinstance(i, torch.Tensor) for i in [points, transformation]):
        transformed_points = torch.matmul(
            transformation,
            torch.cat(
                [points[:, 0:3], torch.ones(size=(points.shape[0], 1))], dim=-1
            ).T,
        ).T
    else:
        raise TypeError("Both inputs should be either np.ndarray or torch.Tensor type.")
    points[:, 0:3] = transformed_points[:, 0:3]
    return points


In [ ]:
# creating moving and fixed inputs
input_pcd = o3d.io.read_point_cloud('/content/FCGF/redkitchen-20.ply')

# create fixed input (points and features)
fixed_xyz = np.array(input_pcd.points)
fixed_feats = np.ones((len(fixed_xyz), 1))

# create moving input (points and features)
moving_xyz = np.array(input_pcd.points)
moving_feats = np.ones((len(fixed_feats), 1))


In [ ]:
# performing the transformation on the moving input
random_transform = np.array(
    [[0.57620317, 0.68342775, -0.44823694, 2.4],
     [0.20656687, -0.65240175, -0.729179, 1.0],
     [-0.7907718, 0.3275644, -0.5170895, 3.9],
     [0., 0., 0., 1.]]
)

moving_xyz = apply_transformation(moving_xyz, random_transform)

In [ ]:
# create point cloud objects
fixed_pcd = o3d.geometry.PointCloud()
fixed_pcd.points = o3d.utility.Vector3dVector(fixed_xyz)

moving_pcd = o3d.geometry.PointCloud()
moving_pcd.points = o3d.utility.Vector3dVector(moving_xyz)

# visualize the point clouds
o3d.visualization.draw_geometries([fixed_pcd, moving_pcd])

# Visualize down-sampled point cloud
o3d.visualization.draw_plotly([fixed_pcd, moving_pcd], point_sample_factor=0.1)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!python /content/FCGF/demo_alignment.py

/content/FCGF/demo_alignment.py:216: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coordinates=torch.tensor(fixed_coords, dtype=torch.int32),
/content/FCGF/demo_alignment.py:229: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coordinates=torch.tensor(moving_coords, dtype=torch.int32),


In [ ]:
# load the saved PLY files
fixed_ply = o3d.io.read_point_cloud('/content/fixed.ply')
moving_ply = o3d.io.read_point_cloud('/content/moving.ply')

voxel_size = 0.01 # set the voxel size to control the downsampling level
fixed = fixed_ply.voxel_down_sample(voxel_size)
moving = moving_ply.voxel_down_sample(voxel_size)

# further downsample the point clouds to reduce the number of points
fixed = fixed.uniform_down_sample(every_k_points=10)
moving = moving.uniform_down_sample(every_k_points=10)

# extract the point coordinates
fixed_coords = np.asarray(fixed.points)
moving_coords = np.asarray(moving.points)

# create Plotly traces for the point clouds
fixed_trace = go.Scatter3d(
    x=fixed_coords[:, 0],
    y=fixed_coords[:, 1],
    z=fixed_coords[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color='blue',
    ),
    name='Fixed'
)

moving_trace = go.Scatter3d(
    x=moving_coords[:, 0],
    y=moving_coords[:, 1],
    z=moving_coords[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color='red',
    ),
    name='Moving'
)

# create a Plotly figure with the traces
fig = go.Figure(data=[fixed_trace, moving_trace])
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=1)))

# show the Plotly figure
fig.show()

In [ ]:
# load the saved PLY files
fixed_ply = o3d.io.read_point_cloud('/content/fixed_aligned.ply')
moving_ply = o3d.io.read_point_cloud('/content/moving_aligned.ply')

voxel_size = 0.01 # set the voxel size to control the downsampling level
fixed = fixed_ply.voxel_down_sample(voxel_size)
moving = moving_ply.voxel_down_sample(voxel_size)

# further downsample the point clouds to reduce the number of points
fixed = fixed.uniform_down_sample(every_k_points=10)
moving = moving.uniform_down_sample(every_k_points=10)

# extract the point coordinates
fixed_coords = np.asarray(fixed.points)
moving_coords = np.asarray(moving.points)

# create Plotly traces for the point clouds
fixed_trace = go.Scatter3d(
    x=fixed_coords[:, 0],
    y=fixed_coords[:, 1],
    z=fixed_coords[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color='blue',
    ),
    name='Fixed'
)

moving_trace = go.Scatter3d(
    x=moving_coords[:, 0],
    y=moving_coords[:, 1],
    z=moving_coords[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color='red',
    ),
    name='Moving'
)

# create a Plotly figure with the traces
fig = go.Figure(data=[fixed_trace, moving_trace])
fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=1)))

# show the Plotly figure
fig.show()

In [ ]:
# Data set for the training
!chmod +x ./scripts/download_datasets.sh
!./scripts/download_datasets.sh /path/to/dataset/download/dir

/path/to/dataset/download/dir
#################################
Data Root Dir: /path/to/dataset/download/dir
Download Path: /path/to/dataset/download/dir/tmp
#################################
>> Create temporary directory: /path/to/dataset/download/dir/tmp
>> Start downloading
threedmatch.tgz     100%[===================>]   7.57G  15.8MB/s    in 8m 27s  
>> Unpack .zip file
./threedmatch/
./threedmatch/bundlefusion-apt0@seq-01_005.npz
./threedmatch/rgbd-scenes-v2-scene_02@seq-01_004.npz
./threedmatch/sun3d-home_bksh-home_bksh_oct_30_2012_scan2_erika@seq-01_086.npz
./threedmatch/sun3d-mit_76_417-76-417b@seq-01_002.npz
./threedmatch/rgbd-scenes-v2-scene_05@seq-01_007.npz
./threedmatch/bundlefusion-office1@seq-01_039.npz
./threedmatch/bundlefusion-office0@seq-01_004.npz
./threedmatch/sun3d-home_md-home_md_scan9_2012_sep_30@seq-01_032.npz
./threedmatch/7-scenes-office@seq-05_000.npz
./threedmatch/sun3d-mit_w20_athena-sc_athena_oct_29_2012_scan1_erika@seq-01_011.npz
./threedmatch/sun3d-mit